# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 30 2022 8:57AM,240808,12,619567,"NBTY Global, Inc.",Released
1,Jun 30 2022 8:57AM,240808,12,619568,"NBTY Global, Inc.",Released
2,Jun 30 2022 8:51AM,240806,10,SO94302,"Senseonics, Incorporated",Released
3,Jun 30 2022 8:51AM,240806,10,SO94308,"Senseonics, Incorporated",Released
4,Jun 30 2022 8:51AM,240805,16,SSF-8212512,Sartorius Stedim Filters Inc.,Released
5,Jun 30 2022 8:50AM,240804,19,TR-RIH-07012022,"GCH Granules USA, Inc.",Released
6,Jun 30 2022 8:50AM,240804,19,TR-RG-07062022,"GCH Granules USA, Inc.",Released
7,Jun 30 2022 8:50AM,240804,19,TR-RO-06292022,"GCH Granules USA, Inc.",Released
8,Jun 30 2022 8:46AM,240803,10,0085814316,ISDIN Corporation,Released
9,Jun 30 2022 8:46AM,240803,10,0085814392,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
26,240803,Released,29
27,240804,Released,3
28,240805,Released,1
29,240806,Released,2
30,240808,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
240803,NaN,NaN,29.0
240804,NaN,NaN,3.0
240805,NaN,NaN,1.0
240806,NaN,NaN,2.0
240808,NaN,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240745,1.0,7.0,13.0
240750,0.0,16.0,10.0
240751,0.0,0.0,1.0
240763,1.0,0.0,5.0
240766,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240745,1,7,13
240750,0,16,10
240751,0,0,1
240763,1,0,5
240766,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240745,1,7,13
1,240750,0,16,10
2,240751,0,0,1
3,240763,1,0,5
4,240766,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240745,1,7,13
1,240750,,16,10
2,240751,,,1
3,240763,1,,5
4,240766,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 30 2022 8:57AM,240808,12,"NBTY Global, Inc."
2,Jun 30 2022 8:51AM,240806,10,"Senseonics, Incorporated"
4,Jun 30 2022 8:51AM,240805,16,Sartorius Stedim Filters Inc.
5,Jun 30 2022 8:50AM,240804,19,"GCH Granules USA, Inc."
8,Jun 30 2022 8:46AM,240803,10,ISDIN Corporation
37,Jun 30 2022 8:26AM,240801,16,Sartorius Bioprocess Solutions
44,Jun 30 2022 7:47AM,240800,19,"AdvaGen Pharma, Ltd"
98,Jun 30 2022 7:34AM,240798,19,"Innogenix, LLC"
113,Jun 29 2022 3:33PM,240786,21,"NBTY Global, Inc."
114,Jun 29 2022 3:32PM,240785,21,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 30 2022 8:57AM,240808,12,"NBTY Global, Inc.",,,2
1,Jun 30 2022 8:51AM,240806,10,"Senseonics, Incorporated",,,2
2,Jun 30 2022 8:51AM,240805,16,Sartorius Stedim Filters Inc.,,,1
3,Jun 30 2022 8:50AM,240804,19,"GCH Granules USA, Inc.",,,3
4,Jun 30 2022 8:46AM,240803,10,ISDIN Corporation,,,29
5,Jun 30 2022 8:26AM,240801,16,Sartorius Bioprocess Solutions,,1,6
6,Jun 30 2022 7:47AM,240800,19,"AdvaGen Pharma, Ltd",,1,53
7,Jun 30 2022 7:34AM,240798,19,"Innogenix, LLC",,13,2
8,Jun 29 2022 3:33PM,240786,21,"NBTY Global, Inc.",,,1
9,Jun 29 2022 3:32PM,240785,21,"NBTY Global, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 30 2022 8:57AM,240808,12,"NBTY Global, Inc.",2,,
1,Jun 30 2022 8:51AM,240806,10,"Senseonics, Incorporated",2,,
2,Jun 30 2022 8:51AM,240805,16,Sartorius Stedim Filters Inc.,1,,
3,Jun 30 2022 8:50AM,240804,19,"GCH Granules USA, Inc.",3,,
4,Jun 30 2022 8:46AM,240803,10,ISDIN Corporation,29,,
5,Jun 30 2022 8:26AM,240801,16,Sartorius Bioprocess Solutions,6,1,
6,Jun 30 2022 7:47AM,240800,19,"AdvaGen Pharma, Ltd",53,1,
7,Jun 30 2022 7:34AM,240798,19,"Innogenix, LLC",2,13,
8,Jun 29 2022 3:33PM,240786,21,"NBTY Global, Inc.",1,,
9,Jun 29 2022 3:32PM,240785,21,"NBTY Global, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 30 2022 8:57AM,240808,12,"NBTY Global, Inc.",2,,
1,Jun 30 2022 8:51AM,240806,10,"Senseonics, Incorporated",2,,
2,Jun 30 2022 8:51AM,240805,16,Sartorius Stedim Filters Inc.,1,,
3,Jun 30 2022 8:50AM,240804,19,"GCH Granules USA, Inc.",3,,
4,Jun 30 2022 8:46AM,240803,10,ISDIN Corporation,29,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 30 2022 8:57AM,240808,12,"NBTY Global, Inc.",2.0,NaN,NaN
1,Jun 30 2022 8:51AM,240806,10,"Senseonics, Incorporated",2.0,NaN,NaN
2,Jun 30 2022 8:51AM,240805,16,Sartorius Stedim Filters Inc.,1.0,NaN,NaN
3,Jun 30 2022 8:50AM,240804,19,"GCH Granules USA, Inc.",3.0,NaN,NaN
4,Jun 30 2022 8:46AM,240803,10,ISDIN Corporation,29.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 30 2022 8:57AM,240808,12,"NBTY Global, Inc.",2.0,0.0,0.0
1,Jun 30 2022 8:51AM,240806,10,"Senseonics, Incorporated",2.0,0.0,0.0
2,Jun 30 2022 8:51AM,240805,16,Sartorius Stedim Filters Inc.,1.0,0.0,0.0
3,Jun 30 2022 8:50AM,240804,19,"GCH Granules USA, Inc.",3.0,0.0,0.0
4,Jun 30 2022 8:46AM,240803,10,ISDIN Corporation,29.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,963126,33.0,0.0,0.0
12,240808,2.0,0.0,0.0
16,481606,7.0,1.0,0.0
17,240768,1.0,0.0,0.0
19,963171,62.0,14.0,0.0
20,1444555,40.0,42.0,4.0
21,1203910,4.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,963126,33.0,0.0,0.0
1,12,240808,2.0,0.0,0.0
2,16,481606,7.0,1.0,0.0
3,17,240768,1.0,0.0,0.0
4,19,963171,62.0,14.0,0.0
5,20,1444555,40.0,42.0,4.0
6,21,1203910,4.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,33.0,0.0,0.0
1,12,2.0,0.0,0.0
2,16,7.0,1.0,0.0
3,17,1.0,0.0,0.0
4,19,62.0,14.0,0.0
5,20,40.0,42.0,4.0
6,21,4.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,33.0
1,12,Released,2.0
2,16,Released,7.0
3,17,Released,1.0
4,19,Released,62.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,17,19,20,21
Status,,,,,,,
Completed,0.0,0.0,0.0,0.0,0.0,4.0,0.0
Executing,0.0,0.0,1.0,0.0,14.0,42.0,1.0
Released,33.0,2.0,7.0,1.0,62.0,40.0,4.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,17,19,20,21
0,Completed,0.0,0.0,0.0,0.0,0.0,4.0,0.0
1,Executing,0.0,0.0,1.0,0.0,14.0,42.0,1.0
2,Released,33.0,2.0,7.0,1.0,62.0,40.0,4.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,17,19,20,21
0,Completed,0.0,0.0,0.0,0.0,0.0,4.0,0.0
1,Executing,0.0,0.0,1.0,0.0,14.0,42.0,1.0
2,Released,33.0,2.0,7.0,1.0,62.0,40.0,4.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()